In [ ]:
from openai import OpenAI
import gpt as gpt
import json
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w', encoding='utf-8') as fp:
        json.dump(data, fp, indent=4)

# openai
openai_api_key = open("openai_api_key").read()
openai_client=OpenAI(api_key=openai_api_key)
def messages_to_string(messages, pid):
    messages_str = ""
    for index, message in enumerate(messages):
        messages_str += str(index) + ". "
        if message['speaker'] == "1":
            messages_str += "採訪者: " + message['content'] + "\n"
        elif message['speaker'] == "0":
            messages_str += pid + ": " + message['content'] + "\n"
    return messages_str


In [ ]:
from pprint import pprint
def speaker_to_str(speaker, pid):
    if speaker == "1":
        return "採訪者"
    elif speaker == "0":
        return pid
    else:
        return pid
def collect_messages(messages, pid):
    result = []
    for index, message in enumerate(messages):
        if message['speaker'] == 1:
            previous_message, next_message = "", ""
            if index > 0:
                previous_message = speaker_to_str(messages[index-1]['speaker'], pid) + messages[index - 1]['content'] + "\n"
            if index < len(messages) - 1:
                next_message = speaker_to_str(messages[index+1]['speaker'], pid) + messages[index + 1]['content'] + "\n"
            current_message = speaker_to_str(message['speaker'], pid) + message['content'] + "\n"
            current_message = "<target>" + current_message + "</target>\n"
            result.append(previous_message + current_message + next_message)
    return result
def classify_interviewer_message_prompt(interviewer_messages):
    # conversations_str = messages_to_string(interviewer_messages, pid)
    messages = [
        {
            "role": "system",
            "content": """你是一個訪談紀錄的分類系統。現在有一個很長的訪談紀錄，需要你幫忙分類。
                用戶的需求是：在訪談紀錄中，有些採訪者的發言是引導新的討論，有些採訪者的發言只是重複之前的討論。
                通常，如果發言很長或是有提到具體的名詞，就是引導新的討論.
                如果發言很短且只是語助詞，比如‘真的喔？’，‘的確’， ‘有趣’，就不太可能是在引導新的討論.
                請根據上下文，判斷被wrap在<target></target>中的採訪者發言是否在引導新的討論，用yes or no來回答。
                Please respond with the following JSON format:
                {{
                    "answer": yes or no
                }}
        """
        },
        {
            "role": "user",
            "content": interviewer_messages
        }
    ]
    return messages
def collect_interviewee_messages(transcripts, pid):
    # 0: interviewee, 1: interviewer
    # speakers = [0 if t['speaker'] == pid else 1 for t in transcripts]
    speakers = [t['speaker'] for t in transcripts]
    # print("speakers:", speakers)
    qa_pairs = []
    i = 0
    cur_q = []
    while i < len(speakers):
        if i == 0:
            cur_q.append(i)
            i += 1
            continue
        if speakers[i] == 1: # interviewer
            qa_pairs.append(cur_q)
            cur_q = [i]
            i += 1
        else: # interviewee
            cur_q.append(i)
            i += 1
    qa_pairs.append(cur_q)
    return qa_pairs
def qa_index_to_message(qa_pairs, transcripts):
    res = []
    for qa in qa_pairs:
        res.append([transcripts[i]['content'] for i in qa])
    return res
def classify_interviewer_message(question):
    messages = [
        # {
        #     'role': 'system',
        #     'content': """
        #         The user is dealing with a transcript of an interview.
        #         In the transcript, some interviewer messages are just repeating what the interviewee said, these messages are less interesting.
        #         Others are for invoking a new discussion, or asking about a new topic in the interview.
        #         You are a classification system that helps user decide if a sentence is invoking a new topic.
        #         Normally, if the sentence is long it should be yes.
        #         Reply with "yes" if the sentence is worth further examination, otherwise reply with "no".
        #     """
        # },
        {
            'role': 'system',
            'content': """
                你是一個訪談紀錄的分類系統。現在有一個很長的訪談紀錄，需要你幫忙分類。
                用戶的需求是：在訪談紀錄中，有些發言是引導新的討論，有些發言只是重複之前的討論。
                通常，如果發言很長或是有提到具體的名詞，就是引導新的討論, 請回答「是」。
                如果發言很短且只是語助詞，比如‘真的喔？’，‘的確’， ‘有趣’，就不太可能是在引導新的討論，請回答「否」。
                請判斷一個發言是否在引導新的討論，用「是」或「否」來回答。
            """
        },
        # example 1
        {
            'role': 'system',
            'name': 'example_user',
            'content': '問：這麼多喔。'

        },
        {
            'role': 'system',
            'name': 'example_system',
            'content': '否'
        },
        # example 2
        {
            'role': 'system',
            'name': 'example_user',
            'content': """
            問：的確，我們這邊就是社會，目前也還沒有說真的要針對哪一塊，但是我們是想要聊解現在這個狀況，就是綠島人民來說就是居民來說不就新居民就居民，他們現在重視有什麼樣的議題。\n
            """

        },
        {
            'role': 'system',
            'name': 'example_system',
            'content': '是'
        },
        # example 3
        {
            'role': 'system',
            'name': 'example_user',
            'content': """
            問：那您就是您覺得您在就是綠島工作，有沒有因為是綠島的緣故，就是讓您工作比較有挑戰性的地方？或者是有，就是特別特別喜歡的地方?\n
            """

        },
        {
            'role': 'system',
            'name': 'example_system',
            'content': '是'
        },
        {
            'role': 'user',
            'content': "問：{}".format(question)
        }
    ]

    return gpt.request_chatgpt_gpt4(openai_client, messages)

In [ ]:
import glob
from pprint import pprint
# chunk_prompts = []
for transcript in glob.glob("raw/*.json"):
    print(transcript)
    pid = transcript.replace(".json", "").replace("raw/", "")
    transcript = json.load(open(transcript))
    # print(transcript)
    res = []
    print(pid)
    qa_pairs = collect_interviewee_messages(transcript, pid)
    qa_pairs = qa_index_to_message(qa_pairs, transcript) 
    print(qa_pairs)
    for qa_index, qa_pair in enumerate(qa_pairs):
        if len(qa_pair) == 1:
            qa_pair = ['（訪談開始）', qa_pair[0]]
            qa_pairs[qa_index] = qa_pair
        if len(qa_pair[0]) < 50 and qa_index > 0 and len(qa_pairs[qa_index-1][1]) < 30:
            label = '否'
        else:
            label = classify_interviewer_message(qa_pair[0])
            if label not in ['是', '否']:
                label = classify_interviewer_message(qa_pair[0])
                if label not in ['是', '否']:
                    label = '否'
        print(label, len(qa_pair[0]), len(qa_pairs[max(0, qa_index-1)][1]),qa_pair)
        res.append({
            'sentence': qa_pair[0],
            'label': label
        })
    save_json(res, 'clf/{}.json'.format(pid))

In [ ]:
def chunk_by_topic(qa_messages, speakers, interviewer_clf):
    index = 0
    interviewer_index = 0
    chunks = []
    new_chunk = []
    for qa_index, qa in enumerate(qa_messages):
        for message in qa:
            if speakers[index] == 1:
                new_chunk.append({
                    "speaker": 1,
                    "content": message
                })
                interviewer_index += 1
            else:
                new_chunk.append({
                    "speaker": 0,
                    "content": message
                })
            if qa_index < len(qa_messages)-1 and speakers[index+1] == 1 and interviewer_clf[interviewer_index] == 1:
                chunks.append(new_chunk)
                new_chunk = []
            index += 1
    chunks.append(new_chunk)
    return chunks

In [ ]:
for transcript in glob.glob("raw/*.json"):
    print(transcript)
    pid = transcript.replace(".json", "").replace("raw/", "")
    transcript = json.load(open(transcript))
    speakers = [t['speaker'] for t in transcript]
    intvwer_clf = json.load(open('clf/{}.json'.format(pid)))
    intvwer_clf = [1 if i['label'] == '是' else 0 for i in intvwer_clf]

    qa_pairs = collect_interviewee_messages(transcript, pid)
    qa_pairs = qa_index_to_message(qa_pairs, transcript) 
    chunks = chunk_by_topic(qa_pairs, speakers, intvwer_clf) 
    save_json(chunks, 'chunks/{}.json'.format(pid))


In [ ]:
# reformat
for chunk_file in glob.glob("chunks/*.json"):
    participant_id = chunk_file.replace(".json", "").replace("chunks/", "")
    chunks = json.load(open(chunk_file))
    reformat_chunks = []
    for chunk_index, chunk in enumerate(chunks):
        reformat_chunks.append({
            "id": "{}_{}".format(participant_id, chunk_index),
            "conversation": chunk,
        })
    save_json(reformat_chunks, 'chunk_summaries/{}.json'.format(participant_id))

In [ ]:
def toString(chunk):
    result = ''
    for dialogue in chunk:
        if dialogue['speaker'] == '1':
            result += "Interviewer: "
        else:
            result += "Interviewee: "
        # result += dialogue['speaker'] + ": " + dialogue['content'] + '\n'
        result += dialogue['content'] + "\n"
    return result

def extract_keywords(paragraph):
    messages = [
        {
            "role": "system",
            "content": """You are a keyword extraction system that extracts keywords from a monologue. 
            The monologue is about a person who is talking about their life.
            The keywords should be the most important words in the monologue.
            Use the exact words that the person uses in the monologue.
            Replay with a list of keywords in Traditional Chinese in the following JSON format:
            {{
                "keywords": ["keyword1", "keyword2", "keyword3", ...]
            }}
            """
       },
       {
           "role": "user",
           "content": paragraph
       }
    ]
    # keywords = request_chatgpt_gpt4(messages)
    return messages
def extract_title(conversation):
    messages = [
        {
            "role": "system",
            "content": """You are a title analysis system. 
            You are given a conversation between two people: Interviewer and Interviewee. 
            Give a concise title for the conversation with no more than 10 words.
            Reply with Traditional Chinese.
        """
        },
        {
            "role": "user",
            "content": conversation
        }
    ]
    return messages

def topic_analysis(text, topics):
    messages = [
        {
            "role": "system",
            "content": """You are a topic analysis system. 
            You are given a conversation between two people: Interviewer and Interviewee. 
            The conversation is mostly about the environment, but the user cares more about a specific aspect.
            What is the focus aspect of the conversation?
            Reply with exactly only one of the following choices: {}
            Be aware that the user wants to distinguish between water/ocean environment and land environment.
            Reply with the following JSON format in Traditional Chinese:
            {{ "result": string (from one of the provided choices) }}
        """.format(", ".join(topics))
        },
        {
            "role": "user",
            "content": text
        }
    ]
    return messages
def emotion_analysis(text, emotions):
    messages = [
        {
            "role": "system",
            "content": """You are a emotion analysis system. 
            You are given a conversation between two people: Interviewer and Interviewee. 
            What is the overall emotion of the Interviewee?
            Reply with exactly only one of the following emotions: {}
            Reply with the following JSON format:
            {{ "emotion": string (from one of the provided choices) }}
        """.format(", ".join(emotions))
        },
        {
            "role": "user",
            "content": "Conversation: " + text
        }
    ]
    return messages


In [ ]:
import glob
topics = ['農業','商業', '觀光', '交通','整體經濟','能源','災害','貿易','政府運作','住屋','醫療','公有土地', '其他', '垃圾處理' , '汙染' , '海域生態環境', '陸地生態環境']
emotions = ["proud", "resigned", "angry", "worried", "neutral"]
for interview_file in glob.glob("chunk_summaries/*.json"):
    pid = interview_file.replace(".json", "").replace("chunk_summaries/", "")
    interview_data = json.load(open(interview_file))
    print(interview_file)
    keyword_prompts = []
    title_prompts = []
    topic_prompts = []
    emotion_prompts = []
    for chunk in interview_data:
        interviewee_messages = "\n".join([message['content'] for message in chunk['conversation'] if message['speaker'] == "0"])
        chunk_messages = toString(chunk['conversation'])
        keyword_prompts.append(extract_keywords(interviewee_messages))
        title_prompts.append(extract_title(chunk_messages))
        topic_prompts.append(topic_analysis(chunk_messages, topics))
        emotion_prompts.append(emotion_analysis(chunk_messages, emotions))

    chunk_keywords = gpt.multithread_prompts(openai_client, keyword_prompts)
    chunk_titles = gpt.multithread_prompts(openai_client, title_prompts)
    chunk_topics = gpt.multithread_prompts(openai_client, topic_prompts, format="json")
    chunk_emotions = gpt.multithread_prompts(openai_client, emotion_prompts, format="json")
    result = []
    for chunk, keywords, title, topic, emotion in zip(interview_data, chunk_keywords, chunk_titles, chunk_topics, chunk_emotions):
        chunk_messages = toString(chunk['conversation'])
        chunk['raw_keywords'] = keywords
        chunk['title'] = title
        # topic
        topic_max_retry = 3
        try:
            topic = json.loads(topic)['result']
        except:
            topic = "error"
        chunk['topic'] = topic
        # emotion
        emotion_max_retry = 3
        try:
            emotion = json.loads(emotion)['emotion']
        except:
            emotion = "error"
        chunk['emotion'] = emotion
        result.append(chunk)
    save_json(result, "chunk_summaries_w_ktte/{}.json".format(pid))

In [ ]:
import glob 
topics = ['農業','商業','文化', '觀光', '交通','整體經濟','能源','災害','貿易','政府運作','住屋','醫療','公有土地', '其他', '垃圾處理' , '汙染' , '海域生態環境', '陸地生態環境']
emotions = ["proud", "resigned", "angry", "worried", "neutral"]
index = 0
for interview_file in glob.glob("chunk_summaries_w_ktte/*.json"):
    # if index == 0: 
    #     index += 1
    #     continue
    pid = interview_file.replace(".json", "").replace("chunk_summaries_w_ktte/", "")
    chunks = json.load(open(interview_file))
    # topic
    for index, chunk in enumerate(chunks):
        if chunk['topic'] in topics: continue
        retry = 5
        topic = chunk['topic']
        while topic == "error" or topic not in topics:
            topic = gpt.request_chatgpt_gpt4(openai_client, topic_analysis(toString(chunk['conversation']), topics))
            topic = topic.replace("```", "").replace("json", "")
            print("retrying topic...", topic)
            topic = json.loads(topic)['result']
            if topic == "土地生態環境": topic = "陸地生態環境"
            # print(topic, topic in topics)
            retry -= 1
            if retry == 0:
                topic = "其他"
                print("retry out.................")
                break
        chunks[index]['topic'] = topic
    # emotion
    for index, chunk in enumerate(chunks):
        if chunk['emotion'] in emotions: continue
        retry = 3
        while chunk['emotion'] == "error" or chunk['emotion'] not in emotions:
            emotion = gpt.request_chatgpt_gpt4(openai_client, emotion_analysis(toString(chunk['conversation']), emotions))
            print("retrying emotion...", emotion)
            emotion = json.loads(emotion)['emotion']
            retry -= 1
            if retry == 0:
                break
        chunks[index]['emotion'] = emotion
    save_json(chunks, "chunk_summaries_w_ktte/{}.json".format(pid))
    # break
# while topic not in topics and topic_max_retry > 0:
#     topic = json.loads(gpt.request_chatgpt_gpt4(openai_client, topic_analysis(chunk_messages, topics)))['result']
#     topic_max_retry -= 1
#     print("retrying topic...", topic)
# emotion = json.loads(emotion)['emotion']
# while emotion not in emotions and emotion_max_retry > 0:
#     emotion = gpt.request_chatgpt_gpt4(openai_client, emotion_analysis(chunk_messages, emotions))
#     emotion_max_retry -= 1
#     print("retrying emotion...", emotion)

In [ ]:
for interview_file in glob.glob("chunk_summaries_w_ktte/*.json"):
    pid = interview_file.replace(".json", "").replace("chunk_summaries_w_ktte/", "")
    chunks = json.load(open(interview_file))
    # topic
    for index, chunk in enumerate(chunks):
        if chunk['topic'] not in topics: print(chunk['topic'])
    # emotion
    for index, chunk in enumerate(chunks):
        if chunk['emotion'] not in emotions: print(chunk['emotion'])

In [ ]:
import glob
for interview_file in glob.glob("chunk_summaries/*.json"):
    pid = interview_file.replace(".json", "").replace("chunk_summaries/", "")
    interview_data = json.load(open(interview_file))
    chunk_contents = [toString(chunk['conversation']) for chunk in interview_data]
    chunk_embeddings = gpt.multithread_embeddings(openai_client, chunk_contents)
    for chunk, embedding in zip(interview_data, chunk_embeddings):
        chunk['embedding'] = embedding
    save_json(interview_data, "chunk_embeddings/{}.json".format(pid))

In [ ]:
def explain_prompt(keyword):
    messages = [
        {
            "role": "system",
            "content": """You are a dictionary system.
            The user wants to know the meaning of a specific word in the context of environment, ecology, an related topics.
            Reply in Traditional Chinese with the definition of the word.
        """
        },
        {
            "role": "user",
            "content": "請解釋一下「{}」".format(keyword)
        }
    ]
    return messages


In [ ]:
done_file = []

In [ ]:
for interview_file in glob.glob("chunk_summaries_w_ktte/*.json"):
    print(interview_file)
    interview_data = json.load(open(interview_file))
    keyword_prompts = []
    for chunk in interview_data:
        interviewee_messages = "\n".join([message['content'] for message in chunk['conversation'] if str(message['speaker']) == "0"])
        keyword_prompts.append(extract_keywords(interviewee_messages))
    keywords_by_chunk = gpt.multithread_prompts(openai_client, keyword_prompts, format="json")
    index = 0
    for chunk, keywords in zip(interview_data, keywords_by_chunk):
        keywords = json.loads(keywords)['keywords']
        print(keywords)
        interview_data[index]['raw_keywords'] = keywords
        index += 1
    save_json(interview_data, interview_file)
    done_file.append(interview_file)

In [3]:
import glob
import json
for interview_file in glob.glob("chunk_summaries_w_ktte/*.json"):
    print(interview_file)
    interview_data = json.load(open(interview_file))
    for chunk in interview_data:
        print(chunk['raw_keywords'])
    break


chunk_summaries_w_ktte/N6.json
['生活', '綠島人', '生態']
['家', '環境', '生活', '擁有', '地方', '收入', '生活價值', '感受', '海', '重要的人', '歸屬感']
['環境', '人']
['綠島', '潛水', '海洋資源', '打工換宿', '乾淨', '適合居住', '蘭嶼', '硬珊瑚礁', '珊瑚礁', '魚類', '當地文化', '接受度']
['蘭嶼', '原住民', '新住民', '排外', '朋友', '開店', '章魚乾']
['傳統文化', '排外', '移來', '土生土長', '歲數', '民宿業', '競爭', '原住民', '外地人']
['排外', '潛水店', '新的', '舊的', '流言緋語', '生意']
['生活', '感覺', '留言', '蜚語']
['挑戰', '獨立生活', '潛水', '縣市']
['觀光旅遊業', '船班問題', '天氣', '風大', '雨大', '浪大', '客人', '舒服', '船班班次', '不順利']
['船', '航權', '班次', '困擾', '取消', '客人', '規則']
['延後', '原因', '浪高', '停船', '風平浪靜', '說的算', '吵架', '船班', '公司', '取消']
['船班', '公司', '家族企業', '權利']
['漁會', '船公司', '碼頭', '漁獲', '協議', '臺東', '爭吵', '撥款', '居民', '生活經濟']
['地人', '連署', '簽名', '抗議', '綠島', '資訊']
['醫療衛生', '交通', '政府運作', '生態', '總體經濟']
['醫療', '長輩', '臺東', '資源', '車禍', '直升機', '搶救']
['外國人', '騎車', '不熟路', '機車', '交通', '船舶', '飛機班次', '小琉球', '公船', '規定時間', '船公司', '票價', '調整', '班次增加', '小型飛機', '舒服', '浪', '15分鐘', '20分鐘', '選擇']
['船的班次', '公船', '飛機', '來回', '半小時', '加開']
['班次', '票價', '飛機', '綠島

In [ ]:
from collections import defaultdict
keyword_freq = defaultdict(int)
for interview_file in glob.glob("chunk_summaries_w_ktte/*.json"):
    print(interview_file)
    interview_data = json.load(open(interview_file))
    for chunk in interview_data:
        keywords = chunk['raw_keywords']
        for k in keywords:
            keyword_freq[k] += 1

# filter out keywords that only appear once
all_keywords = [k for k in keyword_freq.keys() if keyword_freq[k] > 1]
keyword_statistics = [{"keyword": k, "frequency": keyword_freq[k]} for k in all_keywords]
explain_prompts = [explain_prompt(keyword) for keyword in all_keywords]
explanations = gpt.multithread_prompts(openai_client, explain_prompts)
embeddings = gpt.multithread_embeddings(openai_client, explanations)
result = []
for keyword, explanation, embedding in zip(all_keywords, explanations, embeddings):
    result.append({
        "keyword": keyword,
        "explanation": explanation,
        "embeddings": embeddings
    })
save_json(result, "result/keywords.json")
save_json(keyword_statistics, "result/keyword_statistics.json")


In [41]:
result = []
for keyword, explanation, embedding in zip(all_keywords, explanations, embeddings):
    result.append({
        "keyword": keyword,
        "explanation": explanation,
        "embedding": embedding
    })
save_json(result, "result/keywords.json")
save_json(keyword_statistics, "result/keyword_statistics.json")